# Imports

In [1]:
import numpy as np                     
import pandas as pd
import random as rand
from scipy.signal import correlate, correlation_lags
from scipy.ndimage import gaussian_filter1d

import matplotlib.pyplot as plt       
from matplotlib.patches import FancyArrow, Patch, Circle
import matplotlib.colors as mcolors
from matplotlib.colors import Normalize
import matplotlib.patches as patches
from matplotlib.lines import Line2D

import braingeneers                  
from braingeneers.analysis.analysis import SpikeData, read_phy_files, load_spike_data, burst_detection, randomize_raster
import braingeneers.data.datasets_electrophysiology as ephys
from multiprocessing import Pool
from tqdm import tqdm

# Data

In [2]:
sd = read_phy_files('/workspaces/human_hippocampus/data/ephys/2023-04-02-e-hc328_unperturbed/derived/kilosort2/hc3.28_hckcr1_chip16835_plated34.2_rec4.2_curated.zip')

/tmp/ipykernel_1373/653605344.py:1: DeprecationWarning: Call to deprecated function (or staticmethod) read_phy_files. (Prefer load_spike_data()) -- Deprecated since version 0.1.13.
  sd = read_phy_files('/workspaces/human_hippocampus/data/ephys/2023-04-02-e-hc328_unperturbed/derived/kilosort2/hc3.28_hckcr1_chip16835_plated34.2_rec4.2_curated.zip')


# Code

In [3]:
def calculate_mean_firing_rates(spike_data):
    # Compute mean firing rates for each neuron
    firing_rates = [len(train) / spike_data.length for train in spike_data.train]
    return firing_rates

def get_neuron_positions(spike_data):
    # Extract neuron positions from spike_data
    neuron_x = []
    neuron_y = []
    for neuron in spike_data.neuron_data[0].values():
        neuron_x.append(neuron['position'][0])
        neuron_y.append(neuron['position'][1])
    neuron_positions = np.array([neuron_x, neuron_y]).T
    return neuron_positions

def precalculate_distances_angles(neuron_positions):
    # Vectorized calculation of distances
    diff = neuron_positions[:, np.newaxis, :] - neuron_positions[np.newaxis, :, :]
    distances = np.sqrt(np.sum(diff**2, axis=2))
    
    # Vectorized calculation of angles
    angles = np.arctan2(diff[..., 1], diff[..., 0]) % (2 * np.pi)
    
    return distances, angles


def create_reverse_rank_lookup(event_ranks):
    """
    Create a reverse lookup table for event ranks.
    
    Parameters:
    - event_ranks: A dictionary mapping (neuron_id, spike_time) to event rank.
    
    Returns:
    - A dictionary mapping event rank to (neuron_id, spike_time).
    """
    reverse_lookup = {rank: (neuron_id, spike_time) for (neuron_id, spike_time), rank in event_ranks.items()}
    return reverse_lookup 

def calculate_event_ranks(spike_data, precision=5):
    # Flatten the list of spikes, rounding spike times, and sort by time
    # Include the original index of each spike for uniqueness
    all_spikes = [(neuron_id, round(spike_time, precision), idx) 
                  for neuron_id, spikes in enumerate(spike_data.train) 
                  for idx, spike_time in enumerate(spikes)]
                  
    # Sort by neuron_id, then rounded spike time, then original index
    all_spikes_sorted = sorted(all_spikes, key=lambda x: (x[1], x[0], x[2]))
    
    # Generate a dictionary with event rank as key, (neuron_id, spike_time) as value
    ranks = {rank: (neuron_id, spike_time) for rank, (neuron_id, spike_time, _) in enumerate(all_spikes_sorted)}
    
    print(f"Total unique events: {len(ranks)}")
    
    return ranks

def precompute_close_neurons(distances, window_size=17.5):
    close_neurons = {}
    for i in range(len(distances)):
        close_neurons[i] = [j for j in range(len(distances)) if i != j and distances[i, j] < window_size]
    return close_neurons

def compute_bin_midpoints(bins):
    midpoints = (bins[:-1] + bins[1:]) / 2
    return midpoints

In [5]:
def create_histograms_for_events(spike_data, event_ranks, spatial_range=(82, 1092), time_window_rank=30, bins=6):
    total_events = len(event_ranks)

    histograms_for_each_event = {}

    distances = precalculate_distances_angles(get_neuron_positions(spike_data))[0]
    angles = precalculate_distances_angles(get_neuron_positions(spike_data))[1]

    # distance_bins = np.linspace(0, np.max(distances), bins+1)
    distance_bins = np.linspace(spatial_range[0], spatial_range[1], bins+1)
    angle_bins = np.linspace(0, 2*np.pi, bins+1, endpoint=True)
    # angle_bins[-1] += 1e-10  # Make sure the last bin edge is included

    print_every_n = max(total_events // 100, 1)  # Update progress every 10% or at least once
    
    for current_event_id in event_ranks:
        if current_event_id % print_every_n == 0:
            print(f"Processing event {current_event_id + 1}/{total_events}...")
            
        start_rank = max(0, current_event_id - time_window_rank)
        end_rank = min(total_events, current_event_id + time_window_rank + 1)

        event_distances, event_angles = [], []

        # Only consider events within the rank window
        for other_event_id in range(start_rank, end_rank):
            if other_event_id == current_event_id:
                continue

            current_neuron_id = event_ranks[current_event_id][0]
            other_neuron_id = event_ranks[other_event_id][0]

            # Lookup distance and angle between the two neurons
            distance = distances[current_neuron_id, other_neuron_id]
            angle = angles[current_neuron_id, other_neuron_id]

            if spatial_range[0] < distance < spatial_range[1]:
                event_distances.append(distance)
                event_angles.append(angle)

        event_angles = [0 if angle == 2*np.pi else angle for angle in event_angles]

        distance_hist, _ = np.histogram(event_distances, bins=distance_bins)
        angle_hist, _ = np.histogram(event_angles, bins=angle_bins)

        histograms_for_each_event[current_event_id] = {
            'distance': distance_hist,
            'angle': angle_hist
        }
    
    print("Processing complete")
    return histograms_for_each_event

In [6]:
def apply_sliding_window_average(spike_data, event_histograms, window_width=17.5, bins=6):
    smoothed_histograms = {}
    event_ranks = calculate_event_ranks(spike_data)
    close_neurons = precompute_close_neurons(precalculate_distances_angles(get_neuron_positions(spike_data))[0], window_size=window_width)
    print_every_n = max(len(event_ranks) // 100, 1)  # Update progress every 10% or at least once

    for event_id, histograms in event_histograms.items():
        if event_id % print_every_n == 0:
            print(f"Processing event {event_id + 1}/{len(event_histograms)}...")

        current_neuron_id = event_ranks[event_id][0]

        # Initialize summed histogram for the current event
        summed_histogram = {'distance': np.zeros(bins), 'angle': np.zeros(bins)}
        summed_histogram_count = 1

        # Add the histograms of the current event to the sum
        summed_histogram['distance'] += histograms['distance']
        summed_histogram['angle'] += histograms['angle']

        # Check for close neighbors that occur within 30 ranks
        for other_event_id in close_neurons[current_neuron_id]:
            if abs(event_id - other_event_id) < 30:
                summed_histogram['distance'] += event_histograms[other_event_id]['distance']
                summed_histogram['angle'] += event_histograms[other_event_id]['angle']
                summed_histogram_count += 1
        
        # Compute the average histogram
        if summed_histogram_count > 0:
            smoothed_histograms[event_id] = {
                'distance': summed_histogram['distance'] / summed_histogram_count,
                'angle': summed_histogram['angle'] / summed_histogram_count
            }

    return smoothed_histograms

In [7]:
def subtract_average_histograms(spike_data, averaged_histograms, bins=6):
    final_histograms = {}
    window_histograms = apply_sliding_window_average(spike_data, averaged_histograms, window_width=336, bins=bins)

    final_histograms = {event_id: {
        'distance': np.round(averaged_histograms[event_id]['distance'] - window_histograms[event_id]['distance'], 3),
        'angle': np.round(averaged_histograms[event_id]['angle'] - window_histograms[event_id]['angle'], 3)
    } for event_id in averaged_histograms}

    return final_histograms

In [8]:
def compute_average_angle_from_histogram(angle_hist):
    # Check if the histogram data is valid
    if len(angle_hist) != 6:
        raise ValueError("The histogram must have 6 bins.")

    # Bin midpoints in degrees
    angle_midpoints_degrees = np.array([30, 90, 150, 210, 270, 330])
    # Convert midpoints to radians for calculations
    angle_midpoints_radians = np.radians(angle_midpoints_degrees)

    # Calculate the vector components of each bin's contribution
    x_components = angle_hist * np.cos(angle_midpoints_radians)
    y_components = angle_hist * np.sin(angle_midpoints_radians)

    # Compute the mean vector components
    x_mean = np.sum(x_components) / np.sum(angle_hist)
    y_mean = np.sum(y_components) / np.sum(angle_hist)

    # Compute the arctangent of the mean vector components to get the average angle in radians
    average_angle_radians = np.arctan2(y_mean, x_mean)

    # Normalize the average angle to be within 0 to 2*pi radians
    if average_angle_radians < 0:
        average_angle_radians += 2 * np.pi

    # Return the average angle in radians
    return average_angle_radians

def compute_average_distance_angle(final_histograms, bins=6):
    average_distances_angles = {}

    distance_midpoints = compute_bin_midpoints(np.linspace(82, 1092, bins+1))
    angle_midpoints = compute_bin_midpoints(np.linspace(0, 2*np.pi, bins+1))

    angle_midpoints_degrees = np.degrees(angle_midpoints)

    print(angle_midpoints_degrees)

    for event_id, histograms in final_histograms.items():
        distance_hist = histograms['distance']
        angle_hist = histograms['angle']

        # Compute average distance
        distance_sum = np.sum(distance_hist * distance_midpoints)
        distance_count = np.sum(distance_hist)
        average_distance = distance_sum / distance_count if distance_count != 0 else 0

        # Compute average angle
        average_angle = compute_average_angle_from_histogram(angle_hist)

        # normalized_average_angle = average_angle % (2 * np.pi)

        average_distances_angles[event_id] = {'distance': average_distance, 'angle': average_angle}

    return average_distances_angles

In [20]:
histograms = create_histograms_for_events(sd, calculate_event_ranks(sd), spatial_range=(82, 1092), time_window_rank=30, bins=6)
averaged_histograms = apply_sliding_window_average(sd, histograms, window_width=17.5, bins=6)

Total unique events: 113477
Processing event 1/113477...
Processing event 1135/113477...


/tmp/ipykernel_1373/2426340036.py:10: DeprecationWarning: Call to deprecated function (or staticmethod) neuron_data. (Use NeuronAttributes instead of neuron_data, with the function load_spike_data())
  for neuron in spike_data.neuron_data[0].values():


Processing event 2269/113477...
Processing event 3403/113477...
Processing event 4537/113477...
Processing event 5671/113477...
Processing event 6805/113477...
Processing event 7939/113477...
Processing event 9073/113477...
Processing event 10207/113477...
Processing event 11341/113477...
Processing event 12475/113477...
Processing event 13609/113477...
Processing event 14743/113477...
Processing event 15877/113477...
Processing event 17011/113477...
Processing event 18145/113477...
Processing event 19279/113477...
Processing event 20413/113477...
Processing event 21547/113477...
Processing event 22681/113477...
Processing event 23815/113477...
Processing event 24949/113477...
Processing event 26083/113477...
Processing event 27217/113477...
Processing event 28351/113477...
Processing event 29485/113477...
Processing event 30619/113477...
Processing event 31753/113477...
Processing event 32887/113477...
Processing event 34021/113477...
Processing event 35155/113477...
Processing event 

In [10]:
histograms

{0: {'distance': array([3, 5, 9, 4, 3, 2]),
  'angle': array([ 9, 11,  0,  0,  0,  6])},
 1: {'distance': array([5, 5, 7, 4, 6, 2]),
  'angle': array([ 5,  2,  0,  1, 16,  5])},
 2: {'distance': array([ 9, 10,  1,  4,  4,  1]),
  'angle': array([ 2, 13,  6,  4,  2,  2])},
 3: {'distance': array([4, 9, 7, 1, 5, 2]),
  'angle': array([ 6, 18,  1,  0,  0,  3])},
 4: {'distance': array([2, 3, 4, 3, 3, 7]),
  'angle': array([ 0,  0,  0,  5, 17,  0])},
 5: {'distance': array([9, 7, 8, 5, 1, 4]),
  'angle': array([ 3, 16,  2,  2,  4,  7])},
 6: {'distance': array([5, 6, 7, 2, 7, 4]),
  'angle': array([ 0, 10, 16,  3,  2,  0])},
 7: {'distance': array([ 5, 11,  4,  7,  4,  3]),
  'angle': array([ 0, 15,  9,  5,  4,  1])},
 8: {'distance': array([ 4, 14,  9,  5,  3,  0]),
  'angle': array([ 0, 12,  3,  5, 15,  0])},
 9: {'distance': array([ 2,  6, 12,  9,  3,  4]),
  'angle': array([11,  2,  0,  0, 12, 11])},
 10: {'distance': array([2, 5, 7, 2, 6, 5]),
  'angle': array([ 2,  0,  0,  0, 19,  6]

In [19]:
total_non_whole = 0

# Iterate over the dictionary
for key, value in averaged_histograms.items():
    # Iterate over distance and angle arrays
    for array_name, array in value.items():
        num_not_whole = sum(1 for num in array if num % 1 != 0)
        total_non_whole += num_not_whole

print(f"Total number of elements across all arrays and entries that are not whole numbers or do not end in 0.5: {total_non_whole}")

Total number of elements across all arrays and entries that are not whole numbers or do not end in 0.5: 958
